<a href="https://colab.research.google.com/github/HecMartinSol/03MAIR-Alg.Optimizacion/blob/master/AG3/hector_martin_solis_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3 - Actividad guiada

Héctor Martín Solís

https://colab.research.google.com/drive/1l1VtlepYeVYw2AoOCKWIcpETvrXTFnGJ

In [2]:
!pip install requests

In [3]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f87b3a3a668>)

In [4]:
!pip install tsplib95

In [0]:
import tsplib95
import random
import math

In [0]:
problem = tsplib95.load_problem(file)

In [0]:
Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())

#### RESOLUCION POR BUSQUEDA ALEATORIA

In [0]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion_aleatoria(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

In [9]:
crear_solucion_aleatoria(Nodos)

[0,
 39,
 31,
 28,
 9,
 38,
 18,
 24,
 4,
 36,
 11,
 3,
 6,
 34,
 16,
 8,
 19,
 27,
 29,
 23,
 13,
 1,
 32,
 2,
 5,
 15,
 26,
 25,
 21,
 41,
 22,
 35,
 37,
 7,
 12,
 33,
 14,
 10,
 40,
 20,
 30,
 17]

In [0]:
#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)

In [11]:
distancia(0, 1, problem)

15

In [0]:
#Devuelve la distancia total de una solución establecida
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [0]:
# encuentra la mejor solución entre N iteraciones de soluciones aleatorias
def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion_aleatoria(Nodos)
  mejor_valor = distancia_total(mejor_solucion, problem)
  for i in range(N):
    solucion = crear_solucion_aleatoria(Nodos)
    valor = distancia_total(solucion, problem)
    if valor < mejor_valor:
      mejor_solucion = solucion
      mejor_valor = valor
    
  print("la mejor solucion es", mejor_valor, mejor_solucion)

In [14]:
busqueda_aleatoria(problem,1000)

la mejor solucion es 3941 [0, 32, 14, 5, 1, 2, 9, 33, 22, 40, 39, 13, 16, 26, 18, 15, 37, 17, 34, 6, 10, 38, 41, 7, 25, 19, 36, 20, 31, 12, 4, 23, 8, 29, 11, 28, 21, 24, 27, 3, 35, 30]


-------------------

#### RESOLUCION POR BUSQUEDA LOCAL

In [15]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion
      
solucion = crear_solucion_aleatoria(Nodos)
mejor_solucion = genera_vecina(solucion)

print(mejor_solucion)

[0, 20, 35, 17, 18, 40, 7, 14, 26, 19, 25, 12, 33, 15, 29, 23, 36, 5, 11, 30, 9, 39, 34, 24, 21, 41, 3, 22, 2, 8, 6, 32, 37, 31, 27, 38, 4, 28, 10, 16, 13, 1]


In [0]:
#####################################################################
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion_aleatoria(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina
  
  print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)

In [17]:
busqueda_local(problem, 500)

La mejor solución encontrada es  1827 [0, 5, 19, 13, 26, 29, 39, 24, 40, 21, 9, 23, 41, 10, 18, 17, 36, 35, 31, 6, 4, 28, 30, 32, 20, 33, 34, 38, 22, 8, 25, 11, 12, 14, 16, 15, 37, 7, 1, 3, 2, 27]



--------

#### RESOLUCION POR RECOCIDO SIMULADO

In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

In [23]:
solucion = crear_solucion_aleatoria(Nodos)
mejor_solucion = genera_vecina(solucion)
print(solucion)
print(mejor_solucion)

[0, 19, 37, 18, 29, 40, 23, 21, 38, 20, 25, 6, 39, 32, 13, 31, 4, 14, 33, 8, 41, 10, 11, 7, 5, 34, 12, 26, 35, 2, 9, 17, 1, 27, 16, 15, 24, 36, 3, 28, 30, 22]
[0, 19, 37, 18, 29, 40, 23, 21, 38, 20, 25, 6, 39, 32, 13, 31, 4, 14, 33, 8, 41, 10, 11, 7, 5, 34, 12, 26, 35, 2, 9, 24, 1, 27, 16, 15, 17, 36, 3, 28, 30, 22]


In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False


def bajar_temperatura(T):
  return T*.9999

In [26]:
#####################################################################
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion_aleatoria(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100

  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      # print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=100000)

La mejor solución encontrada es : 1417 [0, 27, 2, 28, 30, 29, 10, 4, 3, 32, 20, 33, 34, 38, 22, 39, 24, 40, 21, 9, 8, 23, 41, 25, 11, 12, 18, 26, 5, 13, 19, 14, 16, 15, 37, 36, 35, 31, 17, 7, 6, 1]


-------

#### RESOLUCION DE COLONIA DE HORMIGAS

In [0]:
import numpy as np
def Add_Nodo(problem, H ,Feromonas) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(list(set(range(1,len(Nodos))) - set(H) )  )

def Incrementa_Feromona(problem, Feromonas, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    Feromonas[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return Feromonas

def Evaporar_Feromonas(Feromonas):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  Feromonas = [[ max(Feromonas[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return Feromonas

In [0]:
def Add_Nodo_Por_Probabilidad(problem, alfa, beta, i, Feromonas, construccion_hormiga) :
  Nodos = list(problem.get_nodes())
  
  J = [x for x in Nodos if x not in construccion_hormiga] # J = nodos no visitados

  # Calculamos el nominador
  nu = [ 0 if distancia(i, j, problem) == 0 else 1 / distancia(i, j, problem) for j in range(len(Nodos))]
  nu_pow_beta = np.power(nu, beta)
  tau_pow_alfa = np.power(Feromonas, alfa)
  nominador = list(np.matmul(tau_pow_alfa, nu_pow_beta))

  # Calculamos el denominador
  tau_to_l = [ 0 if l not in J else Feromonas[i][l] ** alfa for l in range(len(Nodos))]
  nu_to_l = [ 0 if l not in J else nu[l] ** beta for l in range(len(Nodos))]
  denominador = np.sum(tau_to_l) * np.sum(nu_to_l)
  
  # damos probabilidad 0 si el nodo ya se visitó
  prob = [0 if j not in J else nominador[j] / denominador for j in range(len(Nodos))]
  
  # obtenemos el que tenga maxima probabilidad
  nodo_elegido = prob.index(max(prob))
  
  return Nodos[nodo_elegido]

In [0]:
# Colonia de Construccion_de_Hormigas
def Construccion_de_Hormigas(problem, N) :
  #problem = datos del problema
  # N = Número de agentes(hormigas)
    
  Nodos = list(problem.get_nodes())
  Aristas = list(problem.get_edges()) 

  #Inicializa las aristas con una cantidad inicial de feromonas:1
  Feromonas = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Construccion_de_Hormiga = [[0] for _ in range(N)]
  

  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      #Elige el siguiente nodo
      # Nuevo_Nodo = Add_Nodo(problem, Construccion_de_Hormiga[h] ,Feromonas )
      Nuevo_Nodo = Add_Nodo_Por_Probabilidad(problem, 0, 1, i, Feromonas, Construccion_de_Hormiga[h])

      Construccion_de_Hormiga[h].append(Nuevo_Nodo)     
    
    Feromonas = Incrementa_Feromona(problem, Feromonas, Construccion_de_Hormiga[h])
    Feromonas = Evaporar_Feromonas(Feromonas)

  #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Construccion_de_Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Construccion_de_Hormiga[h]
      mejor_distancia = distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)

In [184]:
Construccion_de_Hormigas(problem, 100)

1 1
2 2
3 3
4 4
5 5
6 6
40 40
7 7
8 8
9 9
41 41
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
1 1
2 2
3 3
4 4
5 5
6 6
40 40
7 7
8 8
9 9
41 41
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
1 1
2 2
3 3
4 4
5 5
6 6
40 40
7 7
8 8
9 9
41 41
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
1 1
2 2
3 3
4 4
5 5
6 6
40 40
7 7
8 8
9 9
41 41
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
1 1
2 2
3 3
4 4
5 5
6 6
40 40
7 7
8 8
9 9
41 41
10 10
11 11
12 12
13 13
14 14
15 15
16 1